# Notebook for investigating linearity corrections

Initially written 20 Dec 2021 by Craig Lage\
copying from Chris Waters.

In [ ]:
import sys, os, glob, time
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import astropy.io.fits as pf
from lsst.daf.butler import Butler
import lsst.afw.math as afwMath
from lsst.cp.pipe.utils import (funcPolynomial, irlsFit)

In [ ]:
butler = Butler("/repo/main", collections=["LSSTCam/raw/all","LSSTCam/calib",\
                                                    "u/cslage/calib/13144/calib.20220107"])
camera = butler.get('camera', instrument='LSSTCam')

In [ ]:
linPtcButler = Butler("/repo/main", collections=["u/cslage/bps_13144S"])

In [ ]:
nonlinPtcButler = Butler("/repo/main", collections=["u/cslage/bps_13144M"])

In [ ]:
linButler = Butler("/repo/main", collections=["u/cslage/linearizer_28jan22"])

In [ ]:
lin2Butler = Butler("/repo/main", collections=["u/cslage/linearizer_test_14sep22"])

In [ ]:
def ExpApprox(mu, g, a00, n):
    if (g < 1.0E-6) or (abs(a00) < 1.0E-9):
        return np.zeros([len(mu)])
    else:
        expFactor = 2.0 * a00 * mu * g
        if max(expFactor) > 100.0:
            return np.zeros([len(mu)])
        else:
            preFactor = 1.0 / (2.0 * g * g * a00)
            noiseTerm = n / (g * g)
            return preFactor * (np.exp(expFactor) - 1.0) + noiseTerm
        
def calcMondiode(expId):
    factor = 5.0
    DATA_DIR = '/sdf/group/rubin/lsstdata/offline/instrument/LSSTCam-bot/storage/'
    date = int(expId/100000)
    seq = expId - date * 100000
    date = date - 10000000
    file = DATA_DIR + '%d/MC_C_%d_%06d/Photodiode_Readings_%d_%06d.txt'%(date,date,seq,date,seq)

    x, y = np.recfromtxt(file).transpose()
    # Threshold for finding baseline current values:                                                                                                                                                         
    ythresh = (min(y) + max(y))/factor + min(y)
    # Subtract the median of the baseline values to get a calibrated                                                                                                                                         
    # current.                                                                                                                                                                                               
    y -= np.median(y[np.where(y < ythresh)])
    integral = sum((y[1:] + y[:-1])/2*(x[1:] - x[:-1]))
    return integral


In [ ]:
# Set parameters
minLinearAdu = 2000.0
maxLinearAdu = 20000.0
nSigmaClipLinear = 5.0
fitOrder = 10 # Number of spline knots

In [ ]:
# Find the pdCorrection name:
names = []
for type in list(lin2Butler.registry.queryDatasetTypes()):
    names.append(type.name)
names.sort()
for name in names:
    print(name)

In [ ]:
expId=3021120700200
det = 55
corr = lin2Butler.get('pdCorrection', detector=det, exposure=expId, instrument='LSSTCam')

In [ ]:
expId=3021120700200

names = ["E2V", "ITL"]

correct = "Corrected" # just plot after the photodiode systematic correction
plt.figure(figsize=(16,8))
plt.subplots_adjust(wspace = 0.5, hspace = 0.5)
plotCounter = 1
for i, det in enumerate([55, 74]):
    linPtc = linPtcButler.get('ptc', detector=det, exposure=expId, instrument='LSSTCam')
    nonlinPtc = nonlinPtcButler.get('ptc', detector=det, exposure=expId, instrument='LSSTCam')
    lin = linButler.get('linearizer', detector=det, exposure=expId, instrument='LSSTCam')  # First time  
    lin2 = lin2Butler.get('linearizer', detector=det, exposure=expId, instrument='LSSTCam') # Second time
    for amp in camera[0].getAmplifiers():
        ampName = amp.getName()
        if [det, ampName] not in [[55, 'C17'], [74, 'C01']]:
            continue
        mask = np.array(linPtc.expIdMask[ampName], dtype=bool)
        maxDM = np.max(np.array(linPtc.rawMeans[ampName])[mask])            

        # Now get and plot the linearizer fit
        # This code is copied from cp_pipe/linearity.py
        pdCorrection = linButler.get('pdCorrection', detector=det, exposure=expId, instrument='LSSTCam')
        abscissaCorrections = pdCorrection.abscissaCorrections
        for kk, time in enumerate(["First", "Second"]):
            modExpTimes = []
            for ii, pair in enumerate(linPtc.inputExpIdPairs[ampName]):
                pair = pair[0]
                modExpTime = 0.0
                nExps = 0
                for j in range(2):
                    expId = pair[j]
                    try:
                        monDiode = calcMondiode(expId)
                        modExpTime += monDiode
                        nExps += 1
                    except:
                        continue
                if nExps > 0:
                    # The 5E8 factor bring the modExpTimes back to about the same order as the expTimes                                                                                                          
                    modExpTime = 5.0E8 * modExpTime / nExps
                else:
                    mask[ii] = False


                # Get the photodiode correction                                                                                                                        
                try:
                    correction = abscissaCorrections[str(pair)]
                except:
                    correction = 0.0
                if correct == "Corrected":
                    modExpTimes.append(modExpTime + correction)
                else:
                    modExpTimes.append(modExpTime)

            inputAbscissa = np.array(modExpTimes)[mask]
            if time == "First":
                inputOrdinate = np.array(nonlinPtc.rawMeans[ampName])[mask]
                # Get the spline coordinates from the stored linearizer
                binCenters, values = np.split(lin.linearityCoeffs[amp.getName()], 2)

            else:
                inputOrdinate = np.array(linPtc.rawMeans[ampName])[mask]
                # Get the spline coordinates from the stored linearizer
                binCenters, values = np.split(lin2.linearityCoeffs[amp.getName()], 2)


            fluxMask = inputOrdinate < maxLinearAdu
            lowMask = inputOrdinate > minLinearAdu
            fluxMask = fluxMask & lowMask
            linearAbscissa = inputAbscissa[fluxMask]
            linearOrdinate = inputOrdinate[fluxMask]
            linearFit, linearFitErr, chiSq, weights = irlsFit([0.0, 100.0], linearAbscissa,
                                                              linearOrdinate, funcPolynomial)
            # Convert this proxy-to-flux fit into an expected linear flux
            linearOrdinate = linearFit[0] + linearFit[1] * inputAbscissa

            interp = afwMath.makeInterpolate(binCenters.tolist(), values.tolist(),
                                             afwMath.stringToInterpStyle("AKIMA_SPLINE"))
            modelOrdinate = linearOrdinate + interp.interpolate(linearOrdinate)
            
            maxError = max(abs(values/binCenters))*100
            if time == "First":
                numPlots = 1
            else:
                numPlots = 2
            for ii in range(numPlots):
                plt.subplot(2,3,plotCounter)
                plt.title(f"Spline fit to MonDiode data\n{names[i]} - {det} - {ampName} - {time}\nMaxError = {maxError:.4f} %")
                plt.plot(linearOrdinate, (modelOrdinate-linearOrdinate), ls = '--', lw=3, color='red', label="Spline fit")
                plt.scatter(binCenters, values, marker = 'x', s = 200, color='red', label="Spline knots")
                plt.scatter(linearOrdinate, (inputOrdinate - linearOrdinate), label="Input data")
                if names[i] == 'E2V' and ii == 0:
                    plt.plot([maxDM, maxDM], [-400, 0], ls = '--', color='black', label = "PTC Turnoff")
                    plt.ylim(-500,50)
                elif names[i] == 'ITL' and ii == 0:
                    plt.plot([maxDM, maxDM], [-50, 200], ls = '--', color='black', label = "PTC Turnoff")
                    plt.ylim(-100,200)
                else:
                    plt.plot([maxDM, maxDM], [-25,25], ls = '--', color='black', label = "PTC Turnoff")
                    plt.ylim(-25,25)
                plt.xlabel("Flux (ADU)")
                plt.ylabel("Deviation from Linearity(ADU)")
                plt.xlim(0, 100000)
                plt.xticks([0,25000,50000,75000,100000])
                plt.legend() 
                plotCounter += 1
              

#plt.savefig("/sdf/group/rubin/repo/main/u/cslage/linearizer_test_14sep22/plots/Linearity_Repeat_Test_14Sep22.pdf")

